# Надстройка российского модуля КриптоПро над децентрализованной сетью HyperledgerFabric

### Введение

[**Hyperledger Fabric**](https://www.hyperledger.org/use/fabric) — это программный фреймворк для разработки приложений и специализированных бизнес-решений на основе блокчейна. Это распределенная блокчейн сеть, состоящая из различных функциональных компонентов, которые устанавливаются на узлы сети. Компоненты Hyperledger Fabric представляют из себя Docker контейнеры, которые можно свободно скачать из DockerHub. Hyperledger Fabric также можно запустить в Kubernetes среде.

Для написания смарт-контрактов (chaincode в контексте Hyperledger Fabric) в основном используется Golang  
(хотя Hyperledger Fabric позволяет использовать и другие языки). Для разработки пользовательского приложения чаще всего  
используется Node.js с соответствующим Hyperledger Fabric SDK.


На узлах выполняется бизнес логика **(смарт-контракт) – chaincode**, хранится **состояние распределенного реестра (ledger data)** и исполняются другие системные службы платформы. **Узел – это только логическая единица**, разные узлы могут существовать на одном физическом сервере. Гораздо важнее – это как узлы сгруппированы **(Trusted domain)** и с какими функциями блокчейн сети они ассоциированы.

[**Модуль КриптоПро HLF**](https://www.cryptopro.ru/products/hlf), разработанный на базе сертифицированного СКЗИ КриптоПро CSP, обеспечивает возможность использования **российских криптографических алгоритмов** для реализации функций создания и проверки электронной подписи, шифрования/расшифрования данных в распределённых реестрах на основе Hyperledger Fabric v1.4.

### Установка Hyperledger Fabric

**Требуемые утилиты:**  

1) **Операционная система**: Ubuntu Linux 14.04/16.04 LTS (обе 64bit), или macOS 10.12  
2) **cURL tool**: последняя версия  
3) **git**  
4) **Docker engine**: версия 17.06.2 и выше  
5) **Docker-compose**: версия 1.14 и выше  
6) **Go**: версия 1.13.х  
7) **Node**: версия 8.9 и выше (версия 9 не поддерживается, Node.js версии 10 поддерживается от 10.15.3 и выше)  
8) **npm**: версия 5.х  
9) **Python**: 2.7.x

Возьмём готовый пример децентрализованной сети Hyperledger Fabric, находящийся в открытом доступе:  
**git clone https://github.com/adhavpavan/BasicNetwork-2.0.git**

В рабочей папке создается директория /BasicNetwork-2.0, содержащая все необходимые файлы для запуска сети.  
![jupiter](./hlf.png)

### Установка КриптоПро HLF в сеть

Для того, чтобы использовать российские криптоалгоритмы в новосозданной распределенной сети, внесем некоторые изменения в  
конфигурационные файлы согласно [инструкции](https://www.cryptopro.ru/sites/default/files/products/hlf/cryptopro_hlf_instructions_for_use.pdf)

Для возможности использования поддерживаемых российских и межгосударственных стандартов к репозиторию Hyperledger Fabric v1.4 необходимо применить патч, добавляющий идентификаторы соответствующих алгоритмов из репозитория https://github.com/deemru/fabric/tree/v1.4.1-gost

Далее необходимо добавить соответствующий модуль КриптоПро в чейнкод. Для этого нужно  
добавить пакет "cryptopro" в зависимости *BasicNetwork-2.0/api-1.4/package.json* и импортировать модуль в код чейнкода:
```
"dependencies": {  
  ...  
    "cryptopro": "1.0.0",  
  ...  
}```

Перед использованием необходимо будет ввести серийный номер с бланка лицензии, полученной у организации-разработчика или организации, имеющей права распространения продукта (дилера).  

**cpconfig -license -set <серийный_номер>**

В файлах конфигурационных файлах **BasicNetwork-2.0/artifacts/channel/config/core.yaml** и  
**BasicNetwork-2.0/artifacts/channel/config/order.yaml** необходимо подключить плагин в секции BCCSP:  

```
 Default: PLUGIN
 PLUGIN:  
  Library: /usr/lib/cpro.so  
  Config:  
  Hash: SHA2  
  Security: 256  
  FileKeyStore:  
  KeyStore: /etc/hyperledger/fabric/msp/keystore```  
 
bccsp.BCCSP - плагин, позволяющий использовать российские криптографические алгоритмы, реализованные в СКЗИ КриптоПро CSP

### Сборка

**Установка зависимостей**: *npm install*  
**Развертиывание сети и запуск чейнкода**: *peer chaincode install* - установка чейнкода  
*peer chaincode instantiate* - запуск чейнкода  
**Сборка Hyperledger Fabric v1.4 для использования с плагином BCCSP**: *GOPATH=/go GO_TAGS=pluginsenabled make peer orderer*  
**Запуск**: в директории BasicNetwork-2.0/artifacts необходимо ввести команду *docker-compose up -d*

### Пример использования модуля:

Создадим простейший смарт-контракт в коде чейнкода:

Импортируем модуль:  

```const CryptoPro = require('cryptopro');``` 

Смарт-контракт, реализующий гостовский алгоритм ЭЦП модуля КриптоПро:  

```
const { Context } = require('fabric-contract-api');  
const CryptoPro = require('cryptopro');
```
  
```
class MyContract extends Context {  
  async myFunction() {  
    const cryptoPro = new CryptoPro();  
    const signature = await cryptoPro.sign(data, certificate);  
    return signature;  
  }  
}
```
  
```
module.exports = MyContract;
```